<a href="https://colab.research.google.com/github/Tian-ing/RNN-Application/blob/main/Copy_of_stats_199_rnn_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

load package

In [1]:
pip install sparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 77 kB 3.3 MB/s 


In [2]:
import numpy as np
from scipy.interpolate import BSpline, splrep, UnivariateSpline

from scipy.signal import bspline
import pandas as pd

from sklearn.preprocessing import StandardScaler
import sparse
import time
from scipy.interpolate._bspl import evaluate_all_bspl



generate data

In [3]:
def generate_data(n_patients, n_var, T, idx_x, idx_y, rank):
    Fx = np.random.randn(n_var,rank)
    Fy = np.random.randn(rank)

#     weights = np.random.randn(n_patients, rank, D)        # D is not defined     suppose D as 1;
    x_data = []
    y_data = []
    for i in range(n_patients):
        spl = [lambda t: 0.02*i*np.log(t+1), lambda t: 2*np.exp(-(t-60+10*i)/50*(t-60+10*i)+0.0000001) + 4*np.exp(-(t-70+10*i)/20*(t-70+10*i)+0.0000001) , lambda t: np.cos(0.12*np.pi*t) + 1]
        for j in range(n_var):
            tmp = np.matmul(Fx[j,:], [spl[r](idx_x[i,j,:].data) for r in range(rank)])+ 0.5*np.random.randn(len(idx_x[i,j,:].data))
            x_data = np.concatenate((x_data, tmp))
        tmp = np.matmul(Fy, [spl[r](idx_y[i,0,:].data) for r in range(rank)])+ 0.5*np.random.randn(len(idx_y[i,0,:].data))
        y_data = np.concatenate((y_data, tmp))
    output_x = sparse.COO(idx_x.coords, x_data, shape = (n_patients, n_var, T))
    output_y = sparse.COO(idx_y.coords, y_data, shape = (n_patients, 1, T))

    return [output_x, output_y, Fx, Fy]
############################
I = 33 # # of patients
J = 4 # # of variables for X
#T = 96 # max time
T = 1000
R = 3 # Rank
performance = {}

#    #Same resolution random
#    idx_x = sparse.random((I, J, T), density = 0.7)
#    idx_y_train = sparse.random((I, 1, T), density = 0.7)

#    # Sameresolution same idx
#
#    a = np.random.binomial(1, 0.7, T)
#    idx_x = np.stack(J*[a], axis = 0)
#    idx_x = np.repeat(idx_x.reshape(1,J,T), I, axis=0)
#    idx_x = sparse.COO.from_numpy(idx_x)
#
#    idx_y_train = np.repeat(a.reshape(1,1,T), I, axis=0)
#    idx_y_train = sparse.COO.from_numpy(idx_y_train)
#

############# Multiresolution
idx_x = sparse.random((I, J, T), density = 0.8)
idx_y_train = sparse.random((I, 1, T), density = 0.2)


#    ## Fixed multiresolution
#    a = np.ones((T))
#    b = np.zeros((T))
#    b[range(0, T, 2)] = 1
#    idx_x = np.stack((J-int(J/4))*[a]+int(J/4)*[b], axis = 0)
#    idx_x = np.repeat(idx_x.reshape(1,J,T), I, axis=0)
#    idx_x = sparse.COO.from_numpy(idx_x)
#    b = np.zeros((T))
#    b[range(0, T, 4)] = 1
#    idx_y_train = np.repeat(b.reshape(1,1,T), I, axis=0)
#    idx_y_train = sparse.COO.from_numpy(idx_y_train)
#    b = np.zeros((T))
#    b[range(1, T, 4)] = 1
#    idx_y_test = np.repeat(b.reshape(1,1,T), I, axis=0)
#    idx_y_test = sparse.COO.from_numpy(idx_y_test)


###Idx processing
idx_y_test = np.copy(idx_y_train)
idx_y_test.fill_value = np.nan
idx_y_test = np.isnan(idx_y_test.todense())
idx_y_test = sparse.COO.from_numpy(idx_y_test)
idx_y = idx_y_train+idx_y_test
idx_x.data = idx_x.coords[2,:]
idx_y_train.data = idx_y_train.coords[2,:]
idx_y_test.data = idx_y_test.coords[2,:]
idx_y.data = idx_y.coords[2,:]


[output_x, output_y, trueFx, trueFy] = generate_data(n_patients = I, n_var = J, T = T, idx_x = idx_x, idx_y = idx_y, rank = R)

idx_y_train_copy = np.copy(idx_y_train)
idx_y_train_copy.data = np.ones(len(idx_y_train_copy.data))
output_y_train = output_y * idx_y_train_copy

idx_y_test_copy = np.copy(idx_y_test)
idx_y_test_copy.data = np.ones(len(idx_y_test_copy.data))
output_y_test = output_y * idx_y_test_copy

In [4]:
trueFx

array([[ 0.70415685, -0.23232072,  0.90954434],
       [-2.09320482,  0.16193822, -1.06130033],
       [-0.1539251 ,  2.18717684,  1.58467883],
       [-2.29515695,  0.79661464, -0.41821911]])

In [5]:
output_x

Format,coo
Data Type,float64
Shape,"(33, 4, 1000)"
nnz,105600
Density,0.8
Read-only,True
Size,3.2M
Storage ratio,3.2


In [6]:
print(output_x)

<COO: shape=(33, 4, 1000), dtype=float64, nnz=105600, fill_value=0.0>


In [7]:
import multiprocessing as mp

import itertools
import torch



spl = []
data = []
data1 = []
k = 3
for i in range(I):
    spl.append(UnivariateSpline(idx_y_train[i,0,:].data, output_y_train[i,0,:].data, k = k, s = 200))
    tmp = spl[i](idx_y_train[i,0,:].data)
    data = np.concatenate((data, tmp))
    tmp = spl[i](idx_y_test[i,0,:].data)
    data1 = np.concatenate((data1, tmp))
pred_train = sparse.COO(idx_y_train.coords, data,shape = (I, 1, idx_y_train.shape[2]))
pred_test = sparse.COO(idx_y_test.coords, data1,shape = (I, 1, idx_y_test.shape[2]))

In [8]:
pred_train

Format,coo
Data Type,float64
Shape,"(33, 1, 1000)"
nnz,6600
Density,0.2
Read-only,True
Size,90.2K
Storage ratio,0.3


In [9]:
a = pred_train.todense()
type(a)

numpy.ndarray

In [10]:
a = pred_test.todense()
print(a)

[[[-0.70961316 -0.70654134 -0.70348404 ... -0.08893245 -0.08596065
   -0.0829745 ]]

 [[-0.56968423 -0.56823562 -0.56679017 ...  0.         -0.21669156
    0.        ]]

 [[ 0.         -0.59919481  0.         ... -0.11951353 -0.11953055
   -0.11954558]]

 ...

 [[ 0.          0.          1.71196296 ...  5.19131907  5.19685567
    5.20242084]]

 [[ 2.04431324  2.05683393  2.06931408 ...  5.15617406  5.16070537
    0.        ]]

 [[ 2.07565772  2.08645146  2.09721629 ...  5.47015725  5.4731929
    5.47624193]]]


In [11]:

performance['spline'] =   np.sum((pred_train - output_y_train)**2)/len(pred_train.data)
performance['splinepred'] =   np.sum((pred_test - output_y_test)**2)/len(pred_test.data)
#
###################### DLM
#    assert pyro.__version__.startswith('1.8.1')
#
#    pyro.set_rng_seed(2022)
#
#    pd.set_option('display.max_rows', 500)
#
#    torch.manual_seed(2022)
#
#    class DLM(ForecastingModel):
#        def model(self, zero_data, covariates):
#            data_dim = zero_data.size(-1)
#            feature_dim = covariates.size(-1)
#
#            drift_scale = pyro.sample("drift_scale", dist.LogNormal(-10, 10).expand([feature_dim]).to_event(1))
#            with self.time_plate:
#                with poutine.reparam(config={"drift": LocScaleReparam()}):
#                    drift = pyro.sample("drift", dist.Normal(torch.zeros(covariates.size()), drift_scale).to_event(1))
#
#            weight = drift.cumsum(-2)  # A Brownian motion.
#            # record in model_trace
#            pyro.deterministic("weight", weight)
#
#            prediction = (weight * covariates).sum(-1, keepdim=True)
#            assert prediction.shape[-2:] == zero_data.shape
#            # record in model_trace
#            pyro.deterministic("prediction", prediction)
#
#            scale = pyro.sample("noise_scale", dist.LogNormal(-5, 10).expand([1]).to_event(1))
#            noise_dist = dist.Normal(0, scale)
#
#            self.predict(noise_dist, prediction)
#
#    errorsum = 0
#    errorn = 0
#    errorsum_train = 0
#    errorn_train = 0
#    for i in range(I):
#        y = output_y_train[i,:,:]
#        y.fill_value = 0
#        y = np.transpose(y.todense())
#        y = torch.from_numpy(y)
#
#        covariates = output_x[i,:,:]
#        covariates.fill_value = 0
#        covariates = np.transpose(covariates.todense())
#        covariates = torch.from_numpy(covariates)
#
#        pyro.set_rng_seed(2022)
#        pyro.clear_param_store()
#        model = DLM()
#        forecaster = Forecaster(
#            model,
#            y[:T],
#            covariates[:T],
#            learning_rate=0.1,
#            learning_rate_decay=0.05,
#            num_steps=1000,
#        )
#
#        with poutine.trace() as tr:
#            forecaster(y[:T], covariates[:T], num_samples=100)
#
#        # extract the values from the recorded trace
#        posterior_samples = {
#            name: site["value"]
#            for name, site in tr.trace.nodes.items()
#            if site["type"] == "sample"
#        }
#
#        DLMpred = quantile(posterior_samples['prediction'], 0.5)
#
#        errorsum_train = errorsum_train + np.sum((np.array(DLMpred[:,0])[output_y_train[i,:,:].coords[0]]-output_y_train[i,:,:].data)**2)
#        errorn_train = errorn_train + len(output_y_train[i,:,:].data)
#
#        dlmtest= output_y_test[i,0,:]
#        errorsum = errorsum + np.sum((np.array(DLMpred[:,0])[dlmtest.coords[0]]-dlmtest.data)**2)
#        errorn = errorn + len(dlmtest.data)
#
#    performance['DLM'] =   errorsum_train/errorn_train
#    performance['DLMpred'] =   errorsum/errorn

###################Time series
#    from statsmodels.tsa.api import VAR
#
#    errorsum = 0
#    errorn = 0
#    errorsum_train = 0
#    errorn_train = 0
#
#    for i in range(I):
#        tsdata = np.concatenate((np.transpose(output_x[i,:,:].todense()), np.transpose(output_y_train[i,:,:].todense())),axis=1)
#        model = VAR(tsdata)
#        model_fit = model.fit(9)
#
#        lag_order = model_fit.k_ar
#        TSpred = []
#        startidx = 0
#        for idx in output_y_test[i,0,:].coords[0]:
#            if idx < lag_order:
#                errorn_train = errorn_train - 1
#                startidx += 1
#            else:
#                tmp = model_fit.forecast(tsdata[idx-lag_order:idx], steps=1)
#                tmp = tmp[0,J]
#                TSpred.append(tmp)
#        errorsum = errorsum + np.sum((TSpred-output_y_test[i,0,:].data[startidx:])**2)
#        errorn = errorn + len(output_y_test[i,0,:].data)
#
#        TSpred = []
#        startidx = 0
#        for idx in output_y_train[i,0,:].coords[0]:
#            if idx < lag_order:
#                errorn_train = errorn_train - 1
#                startidx += 1
#            else:
#                tmp = model_fit.forecast(tsdata[idx-lag_order:idx], steps=1)
#                tmp = tmp[0,J]
#                TSpred.append(tmp)
#        errorsum_train = errorsum_train + np.sum((TSpred-output_y_train[i,0,:].data[startidx:])**2)
#        errorn_train = errorn_train + len(output_y_train[i,0,:].data)
#    performance['TS'] =   errorsum_train/errorn_train
#    performance['TSpred'] =   errorsum/errorn
#
#    print(performance)

########RNN
output_y_train.fill_value = 0
output_y_test.fill_value = 0
output_x.fill_value = 0
y0 = np.moveaxis(output_y_train.todense(), -1, 1)
x0 = np.moveaxis(output_x.todense(), -1, 1)
data = np.concatenate((x0, y0), axis=2)



In [12]:
pred_train
type(pred_test)

sparse._coo.core.COO

In [13]:
train_tensor = torch.tensor(pred_train)
test_tensor = torch.tensor(pred_test)


In [14]:
train_tensor.shape

torch.Size([33, 1, 1000])

In [15]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [23]:
# Create RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        return out
    
# batch_size, epoch and iteration
batch_size = 100
n_iters = 10000
num_epochs = n_iters / (len(pred_train) / batch_size)
num_epochs = int(num_epochs)

# Pytorch train and test sets
train = train_tensor
test = test_tensor

# data loader
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)
    
# Create RNN
input_dim = 1000    # input dimension
hidden_dim = 100  # hidden layer dimension
layer_dim = 1     # number of hidden layers
output_dim = 33   # output dimension

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

# Cross Entropy Loss 
error = nn.CrossEntropyLoss()

# SGD Optimizer
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [20]:
from torch.autograd import Variable

In [26]:
seq_dim = 33
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
for epoch in range(num_epochs):
    for i, (features) in enumerate(train_loader):
        festures = features.float()
        train = Variable(features)
        train = train.float()
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(train)
        
        # Calculate softmax and ross entropy loss
        loss = error(outputs, features)
        
        # Calculating gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
        
        if count % 250 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for features in test_loader:
                
                
                # Forward propagation
                outputs = model(features)
                
                # Get predictions from the maximum value
                predicted = torch.max(outputs.data, 1)[1]
                
                # Total number of labels
                total += features.size(0)
                
                correct += (predicted == features).sum()
            
            accuracy = 100 * correct / float(total)
            
            # store loss and iteration
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            if count % 500 == 0:
                # Print Loss
                print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.data[0], accuracy))

RuntimeError: ignored